In [1]:
import pandas as pd
import numpy as np


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\atalb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\atalb\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "c:\Users\atalb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 736, i

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\atalb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\atalb\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "c:\Users\atalb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 736, i

AttributeError: _ARRAY_API not found

In [2]:

users = pd.DataFrame([
    {"user_id": 1, "full_name": "Ryan James", "age": 24, "gender": "M", "zip_code": "85711"},
    {"user_id": 2, "full_name": "Alice Johnson", "age": 30, "gender": "F", "zip_code": "94043"},
    {"user_id": 3, "full_name": "Bob Smith", "age": 23, "gender": "M", "zip_code": "32067"},
])

movies = pd.DataFrame([
    {"movie_id": 101, "movie_title": "Alpha", "release_year": 2020},
    {"movie_id": 102, "movie_title": "Beta", "release_year": 2020},
    {"movie_id": 103, "movie_title": "Gamma", "release_year": 2020},
    {"movie_id": 104, "movie_title": "Delta", "release_year": 2020},
    {"movie_id": 105, "movie_title": "Omega", "release_year": 2021},
])

ratings = pd.DataFrame([
    {"user_id": 1, "item_id": 101, "rating": 5, "timestamp": 1000},
    {"user_id": 1, "item_id": 102, "rating": 3, "timestamp": 1010},
    {"user_id": 2, "item_id": 101, "rating": 4, "timestamp": 1005},
    {"user_id": 2, "item_id": 103, "rating": 5, "timestamp": 1020},
    {"user_id": 3, "item_id": 101, "rating": 5, "timestamp": 1030},
    {"user_id": 3, "item_id": 104, "rating": 4, "timestamp": 1040},
])
ratings = ratings.astype({"user_id": int, "item_id": int, "rating": float, "timestamp": int})


In [6]:
# users
# movies
ratings

,user_id,item_id,rating,timestamp
0,1,101,5.0,1000
1,1,102,3.0,1010
2,2,101,4.0,1005
3,2,103,5.0,1020
4,3,101,5.0,1030
5,3,104,4.0,1040


In [8]:
# ------------------------------
# Helper: check similarity
# ------------------------------

def check_similar_users(ratings_df: pd.DataFrame, user_x: int, user_y: int) -> bool:
    """Return True if two users are similar (per problem spec)."""
    rx = ratings_df[ratings_df["user_id"] == user_x][["item_id", "rating"]].rename(columns={"rating": "r_x"})
    ry = ratings_df[ratings_df["user_id"] == user_y][["item_id", "rating"]].rename(columns={"rating": "r_y"})
    merged = pd.merge(rx, ry, on="item_id", how="inner")
    if merged.empty:
        return False
    merged["abs_diff"] = (merged["r_x"] - merged["r_y"]).abs()
    return merged["abs_diff"].max() == 1.0

In [12]:
def get_recommendations(users_df, movies_df, ratings_df, full_name, year, method) -> str:
    """Return one recommended movie title for the given user, year, and method."""

    # ---------------------------
    # 1. Validate method
    # ---------------------------
    if method not in {"by_popularity", "by_rating", "by_similar_users"}:
        return ""

    # ---------------------------
    # 2. Find the target user
    # ---------------------------
    user_row = users_df[users_df["full_name"] == full_name]
    if user_row.empty:
        return ""
    user_id = int(user_row.iloc[0]["user_id"])

    # ---------------------------
    # 3. Filter movies by year
    # ---------------------------
    movies_year = movies_df[movies_df["release_year"] == year].copy()
    if movies_year.empty:
        return ""

    # ---------------------------
    # 4. Exclude already rated movies
    # ---------------------------
    rated_by_user = set(ratings_df.loc[ratings_df["user_id"] == user_id, "item_id"])
    candidates = movies_year[~movies_year["movie_id"].isin(rated_by_user)]
    if candidates.empty:
        return ""

    # ---------------------------
    # 5a. Method: by_popularity
    # ---------------------------
    if method == "by_popularity":
        counts = (
            ratings_df[ratings_df["item_id"].isin(candidates["movie_id"])]
            .groupby("item_id")
            .size()
            .reset_index(name="count")
        )
        merged = candidates.merge(counts, left_on="movie_id", right_on="item_id", how="left").fillna({"count": 0})
        best = merged.sort_values(["count", "movie_title"], ascending=[False, True])
        return best.iloc[0]["movie_title"]

    # ---------------------------
    # 5b. Method: by_rating
    # ---------------------------
    if method == "by_rating":
        means = (
            ratings_df[ratings_df["item_id"].isin(candidates["movie_id"])]
            .groupby("item_id")["rating"]
            .mean()
            .reset_index(name="avg_rating")
        )
        merged = candidates.merge(means, left_on="movie_id", right_on="item_id", how="left").dropna(subset=["avg_rating"])
        if merged.empty:
            return ""
        best = merged.sort_values(["avg_rating", "movie_title"], ascending=[False, True])
        return best.iloc[0]["movie_title"]

    # ---------------------------
    # 5c. Method: by_similar_users
    # ---------------------------
    if method == "by_similar_users":
        # Find similar users
        other_users = users_df.loc[users_df["user_id"] != user_id, "user_id"]
        similar_users = [u for u in other_users if check_similar_users(ratings_df, user_id, u)]
        if not similar_users:
            return ""

        # Get their candidate ratings
        sim_ratings = ratings_df[
            (ratings_df["user_id"].isin(similar_users)) &
            (ratings_df["item_id"].isin(candidates["movie_id"]))
        ]
        if sim_ratings.empty:
            return ""

        # Pick movie with latest timestamp
        latest = sim_ratings.groupby("item_id")["timestamp"].max().reset_index(name="latest_ts")
        merged = candidates.merge(latest, left_on="movie_id", right_on="item_id")
        best = merged.sort_values(["latest_ts", "movie_title"], ascending=[False, True])
        return best.iloc[0]["movie_title"]

    return ""  # fallback


In [13]:
# ------------------------------
# Example runs
# ------------------------------

print(get_recommendations(users, movies, ratings, "Alice Johnson", 2020, "by_popularity"))
print(get_recommendations(users, movies, ratings, "Alice Johnson", 2020, "by_rating"))
print(get_recommendations(users, movies, ratings, "Alice Johnson", 2020, "by_similar_users"))

Beta
Delta
Delta
